In [1]:
# Alias the module so it can be called as Bar instead of Foo.Bar

alias Foo.Bar, as: Bar

Foo.Bar

In [2]:
# Require the module in order to use its macro

nil

In [3]:
require Foo

CompileError: 1

In [4]:
# Import functions from Fod so they can be called without the `Foo.` prefix
import Foo

CompileError: 1

In [5]:
# Invokes the custom code defined in Foo as an extension point

use Foo

CompileError: 1

We are going to explore them in detail now. Keep in mind the first three are called directives because they have **lexical scope**, while `use` is a common extension point that allows the used module to inject code.

## alias

`alias`  allows you to set up aliaes for any given module name.

Imageine a module uses a specialized list implemented in `Math.List`. The `alias` directive allows referring to `Math.List` just as `List` within the module definition

The original `List` can still be accessed within `Stats` by the fully qualified name `Elixir.List`

In [10]:
defmodule Stats do 
defmodule Wawa do
def t do
1
end
end
end

{:module, Stats, <<70, 79, 82, 49, 0, 0, 3, 68, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 119, 0, 0, 0, 12, 12, 69, 108, 105, 120, 105, 114, 46, 83, 116, 97, 116, 115, 8, 95, 95, 105, 110, 102, 111, 95, 95, 3, ...>>, {:module, Stats.Wawa, <<70, 79, 82, 49, 0, 0, 3, 216, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 126, 0, 0, 0, 13, 17, 69, 108, 105, 120, 105, 114, 46, 83, 116, 97, 116, 115, 46, 87, 97, 119, 97, 8, ...>>, {:t, 0}}}

In [14]:
defmodule TT do

def ll do
alias Stats.Wawa
Wawa.t
end
end


{:module, TT, <<70, 79, 82, 49, 0, 0, 4, 24, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 139, 0, 0, 0, 15, 9, 69, 108, 105, 120, 105, 114, 46, 84, 84, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, 117, 110, ...>>, {:ll, 0}}

In [15]:
TT.ll

1

Alias are frequently used to define shortcuts. In fact, calling `alias` without `:as` option sets the alias automatically to the part of the module name, for example:

In [8]:
alias Math.List

Math.List

Noet that `alias` is **lexically scoped**,  which allows you to set aliass inside specific functions

In [16]:
defmodule Math do 
def plus(a, b) do
alias Math.List
# ...
end

def minus(a, b) do
#...
end
end

{:module, Math, <<70, 79, 82, 49, 0, 0, 4, 88, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 150, 0, 0, 0, 16, 11, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, 102, ...>>, {:minus, 2}}

In the example above, since we are invoking `alias` inside the function `plus/2`, the alias will be valid only inside the function `plus/2`. `minus/2` won't be affected at all.

## require

Elixir provides macros as a mechanism for meta-programming(writing code that generate code). Macros are expanded at compile time.

Public functions in modules are globally available, but in order to use macros, you need to opt-in by requiring the module they are defined in

In [17]:
Integer.is_odd(3)

CompileError: 1

In [18]:
require Integer

Integer

In [21]:
Integer.is_odd(3)

true

## use

The `use` macro is frequently used as an extension point. This means that, when you `use` a module `FooBar`, you allow that module to inject any code in the current module, such as importing itself or other modules, defining new functions, setting a module state, etc

In [23]:
defmodule AssertTest do
use ExUnit.Case, async: true

test "always pass" do
assert true
end
end

RuntimeError: 1

In [25]:
defmodule Feature do
def __using__(t) do
IO.puts "wowowo"
end
end

{:module, Feature, <<70, 79, 82, 49, 0, 0, 4, 64, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 146, 0, 0, 0, 15, 14, 69, 108, 105, 120, 105, 114, 46, 70, 101, 97, 116, 117, 114, 101, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, {:__using__, 1}}

In [26]:
defmodule Example do
use Feature
end

wowowo


{:module, Example, <<70, 79, 82, 49, 0, 0, 3, 72, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 12, 14, 69, 108, 105, 120, 105, 114, 46, 69, 120, 97, 109, 112, 108, 101, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, :ok}

Behind the scenes, `use` requires the given module and then calls the `__using__/1` callback on it allowing the module to inject some code into the current context. 

Generally speaking, the following module:

In [27]:
defmodule Example do
use Feature, option: :value
end

wowowo


{:module, Example, <<70, 79, 82, 49, 0, 0, 3, 72, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 12, 14, 69, 108, 105, 120, 105, 114, 46, 69, 120, 97, 109, 112, 108, 101, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, :ok}

is compiled into

In [28]:
defmodule Example do
require Feature
Feature.__using__(option: :value)
end

wowowo


{:module, Example, <<70, 79, 82, 49, 0, 0, 3, 72, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 12, 14, 69, 108, 105, 120, 105, 114, 46, 69, 120, 97, 109, 112, 108, 101, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, :ok}

Since `use` allows any code to run, we can't really know the side-effects of using a module without reading its documentation. For this reason, `import` and `alias` are often preferred.

## Understanding Aliases

In [29]:
is_atom(String)

true

In [30]:
to_string(String)

"Elixir.String"

In [31]:
:"Elixir.String" == String

true

In [33]:
:"Elixir.String" === String

true

Aliases expand to atoms because in the Erlang VM(and consequently Elixir) modules are always represented by atoms. 

In [34]:
:lists.flatten([1, [2], 3])

[1, 2, 3]

In Elixir, you don't have to define the `Foo` module before being able to define the `Foo.Bar` module, as the language translates all the module names to atoms. You can define arbitrarily-nested modules without defining any module in the chain(e.g., `Foo.Bar.Baz` without defining `Foo` or `Foo.Bar` first)

In [35]:
defmodule T.B.C do
  def m do
   1
   end
   end
   

{:module, T.B.C, <<70, 79, 82, 49, 0, 0, 3, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 121, 0, 0, 0, 13, 12, 69, 108, 105, 120, 105, 114, 46, 84, 46, 66, 46, 67, 8, 95, 95, 105, 110, 102, 111, 95, 95, 9, ...>>, {:m, 0}}

In [36]:
T.B.C.m

1